# google Map을 활용한 지도시각화

출처 : https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

In [1]:
import gmaps
import gmaps.datasets
import gmaps.geojson_geometries

In [19]:
my_api_key = ''
    # 올리기 전에 지우기
gmaps.configure(api_key=my_api_key)

In [20]:
earthquake_df = gmaps.datasets.load_dataset_as_df('earthquakes')
earthquake_df.head()

,latitude,longitude,magnitude
0,65.193300,-149.072500,1.70
1,38.791832,-122.780830,2.10
2,38.818001,-122.792168,0.48
3,33.601667,-116.727667,0.78
4,37.378334,-118.520836,3.64


In [21]:
locations = earthquake_df[['latitude', 'longitude']]
weights = earthquake_df['magnitude']
fig = gmaps.figure(map_type='SATELLITE')
    # 지도를 만들어 주는 figure
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heatmap_layer)
    # heatmap layer 추가

In [22]:
fig # for development purposes only : 무료버전이므로 제한됨

Figure(layout=FigureLayout(height='420px'))

In [23]:
 # 만들고 나서도 변경 가능
fig.map_type = 'HYBRID'
fig.mouse_handling = 'GREEDY'
fig.layout = {
    'width':'1200px'
}

In [27]:
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 8
heatmap_layer.opacity = 0.7

## marker layer

In [32]:
marker_locations = [
    (-34.0, -59.166672),
    (-32.23333, -64.433327),
    (40.166672, 44.133331),
    (51.216671, 5.0833302),
    (51.333328, 4.25)
]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
                             hover_text='test',
                             info_box_content='TEST',
                             label='ab')
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

## symbol layer

In [33]:
df = gmaps.datasets.load_dataset_as_df('starbucks_kfc_uk')

starbucks_df = df[df['chain_name'] == 'starbucks']
starbucks_df = starbucks_df[['latitude', 'longitude']]

starbucks_layer = gmaps.symbol_layer(
    starbucks_df, fill_color='green', stroke_color='green', scale=2
)
fig = gmaps.figure()
fig.add_layer(starbucks_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## geojson layer

In [41]:
gmaps.geojson_geometries.list_geometries()

dict_keys(['countries', 'countries-high-resolution', 'england-counties', 'us-states', 'us-counties', 'india-states', 'brazil-states'])

In [34]:
countries_geojson = gmaps.geojson_geometries.load_geometry('countries')

fig = gmaps.figure()
gini_layer = gmaps.geojson_layer(countries_geojson)
fig.add_layer(gini_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
rows = gmaps.datasets.load_dataset('gini') # 'rows' is a list of tuples
country2gini = dict(rows) # dictionary mapping 'country' -> gini coefficient

In [38]:
from matplotlib.cm import viridis
from matplotlib.colors import to_hex

# We will need to scale the GINI values to lie between 0 and 1
min_gini = min(country2gini.values())
max_gini = max(country2gini.values())
gini_range = max_gini - min_gini

def calculate_color(gini):
    """
    Convert the GINI coefficient to a color
    """
    # make gini a number between 0 and 1
    normalized_gini = (gini - min_gini) / gini_range

    # invert gini so that high inequality gives dark color
    inverse_gini = 1.0 - normalized_gini

    # transform the gini coefficient to a matplotlib color
    mpl_color = viridis(inverse_gini)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

In [39]:
colors = []
for feature in countries_geojson['features']:
    country_name = feature['properties']['name']
    try:
        gini = country2gini[country_name]
        color = calculate_color(gini)
    except KeyError:
        # no GINI for that country: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

In [40]:
fig = gmaps.figure()
gini_layer = gmaps.geojson_layer(
    countries_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
fig.add_layer(gini_layer)
fig

Figure(layout=FigureLayout(height='420px'))